In [1]:
"""
Created on Wed Jan 15 09:28:16 2020
Group 7
@authors : L.D.
"""

'\nCreated on Wed Jan 15 09:28:16 2020\nGroup 7\n@authors : L.D.\n'

In [4]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import numpy as np
import pandas as pd
import math
import operator
import os
import PIL
from PIL import Image
from tqdm import tqdm_notebook as tqdm
import random
import pickle

import shutil
from shutil import copyfile

# keras : librairie de deep learning
import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, InputLayer, ReLU, AveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

# import for transfer learning
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

Using TensorFlow backend.


In [3]:
%run g7_functions_for_models.ipynb

# Lecure avec keras

In [48]:
df_seat_annot = pd.read_csv('C:/Users/Lilian/Documents/Projet_deep_learning/g7_SEATGURU_annotate.csv',sep=";")

In [49]:
# path
project_path = 'C:/Users/Lilian/Documents/Projet_deep_learning/'
seatguru_path = 'Interpromo2020/All Data/ANALYSE IMAGE/IMG SEATGURU/'
hackathon_path = 'C:/Users/Lilian/Documents/Projet_deep_learning/InputsHackathon_1/'
crea_path = project_path + 'G7_SEATGURU/Int/Airbus/'
new_paths = [crea_path + 'data_train', crea_path + 'data_test']
print(crea_path + 'data_train')

# Airbus type
airbus_planes = ['A320', 'A321', 'A330', 'A350', 'A380']

# SEATGURU
# Create a directory for each class
create_dirs_seatguru_type(df_seat_annot, project_path, seatguru_path, crea_path, aircraft_types=airbus_planes,
                          view = 'Int', man = 'Airbus')

# Split train and test
split_train_test_seatguru_type(new_paths=new_paths, path=crea_path, aircraft_types=airbus_planes)

# HACKATHON
# Creation of train and test samples for Hackathon pictures
split_train_test_hack(new_paths, hackathon_path, airbus_planes = ['A320', 'A330', 'A350', 'A380'], del_path = False)

C:/Users/Lilian/Documents/Projet_deep_learning/G7_SEATGURU/Int/Airbus/data_train
C:/Users/Lilian/Documents/Projet_deep_learning/G7_SEATGURU/Int/Airbus/A320
A320: 166 images
C:/Users/Lilian/Documents/Projet_deep_learning/G7_SEATGURU/Int/Airbus/A321
A321: 121 images
C:/Users/Lilian/Documents/Projet_deep_learning/G7_SEATGURU/Int/Airbus/A330
A330: 257 images
C:/Users/Lilian/Documents/Projet_deep_learning/G7_SEATGURU/Int/Airbus/A350
A350: 39 images
C:/Users/Lilian/Documents/Projet_deep_learning/G7_SEATGURU/Int/Airbus/A380
A380: 46 images


In [50]:
def flat_list(list_: list) -> list:

    """

    Transforms a list of lists into a flat list

    """

    return [item for sublist in list_ for item in sublist]

In [51]:
def get_imgs(paths_list: list) -> list:
    """
    From a list of images paths, get a list of PIL images
    """
    imgs_list = [Image.open(project_path + data_path + '/'+ paths_list[i]) for i in range(len(paths_list))]

    return imgs_list

In [52]:
project_path = 'C:/Users/Lilian/Documents/Projet_deep_learning'
data_path = '/G7_SEATGURU/Int'

In [53]:
# Get folders list (one item per aircraft manufacturer)

folders_list = os.listdir(project_path + data_path)  # 2 folders: Airbus and Boeing
print(folders_list)
# Desired aircraft types
Airbus_aircraft_types = ['A320', 'A321', 'A330', 'A350', 'A380']
aircraft_types = list([Airbus_aircraft_types])
# Lists: all images paths and names
all_imgs_list = list()
for k in range(len(folders_list)):
    folder = folders_list[k]
    aircraft_types_man = aircraft_types[k]
    # For each folder (aircraft type), get list of all images names (with path)
    for j in range(len(aircraft_types_man)):
        img_list = os.listdir(project_path + data_path + '/' +folder+ '/'+ aircraft_types_man[j])
        img_list = [folder +'/'+aircraft_types_man[j]  + '/' + img_list[k] for k in range(len(img_list))]
        all_imgs_list.append(img_list)
# Flat list

all_imgs_flat_list = flat_list(all_imgs_list)

['Airbus']


In [54]:
def two_step_generator(classes: list, paths_list: list, imgs_per_class: int, shape: tuple,

                       nb_win: int, greys: bool, nb_to_gen: int, img_gen: ImageDataGenerator) -> list:

    

    """

    From a list of reference images, performs a 2-step transformation to generate new images

    of a chosen shape.

    Step 1: Resizes images keeping height to width ratio, and generates a set of cropped images (sliding windows)

    Step 2: Generates new images from Step 1 images, by applying rotations, zooms and shifts

    

    Parameters:

        classes: list of desired classes, e.g.: ['Airbus', 'Boeing'], or a list of aircraft types

        paths_list: list of images paths

        imgs_per_class: desired number of images per class from Step 1 (resize + crop)

        shape: desired shape (height, width)

        nb_win: number of windows

        greys: True for grey scale, False for colour scale

        nb_to_gen: desired number of images per class from data generator

        img_gen: ImageGenerator object

  

    Out:

        datagen: list of generated arrays (representing images)

  

    """

    

    datawin = list()    

    datagen = list()

    

    for class_ in classes:

        print(class_)

        

        # Images paths list

        class_imgs_path = [paths_list[k] for k in range(len(paths_list)) if class_ in paths_list[k]]



        # Randomly choose images

        class_imgs_subset = np.random.choice(class_imgs_path, size=imgs_per_class, replace=False)



        # Get images

        class_imgs = get_imgs(class_imgs_subset)



        # Step 1: resize and crop on sliding windows

        class_new_imgs = create_windows_imgs(class_imgs, shape=shape, nb_win=nb_win, greys=greys)

        class_new_imgs = np.array(flat_list(class_new_imgs))

        datawin.append(class_new_imgs)

    

        # Step 2: DataGenerator

        class_datagen = datagen_class(class_new_imgs, nb_to_gen, img_gen)

        class_datagen = class_datagen.astype(int)



        datagen.append(class_datagen)

        

    return datawin, datagen

In [55]:
def create_windows_imgs(init_imgs: list, shape: tuple, nb_win:int, greys: bool) -> list:

    """
    Parameters:
        init_imgs: list of reference images
        shape: desired shape (height, width)
        nb_win: desired number of windows
    Out:
        new_imgs_all: list of all new arrays (representing images), created by cropping the reference images
    """
    new_imgs_all = list()
    for k in range(len(init_imgs)):
        img = init_imgs[k]
        new_imgs = create_windows_one_img(shape=shape, img=img, nb_win=nb_win, greys=greys)
        new_imgs_all.append(new_imgs)
    return new_imgs_all

In [56]:
def create_windows_one_img(shape: tuple, img: PIL.Image, nb_win: int, greys: bool) -> list:
    """
    Parameters:
        shape: desired shape (height, width)
        img: reference image
        nb_win: desired number of windows
    Out:
        arrs_crop: arrays (representing images) created by cropping the reference image
    """
    img_arr = np.array(img)
    if greys is True:
        imgs_arr = np.mean(img_arr, axis = 2)
    if img_arr.shape[0] <= img_arr.shape[1]:
        # Choose fixed height, calculate width (to keep height to width ratio), resize
        coef = img_arr.shape[0] / shape[0]
        temp_width = int(img_arr.shape[1] / coef)
        resized_img = img.resize((temp_width, shape[0]), Image.BILINEAR)
    else:
        # Choose fixed width, calculate height (to keep height to width ratio), resize
        coef = img_arr.shape[1] / shape[1]
        temp_height = int(img_arr.shape[0] / coef)
        resized_img = img.resize((shape[1], temp_height), Image.BILINEAR)
    # Convert resized image to array
    if greys is True:
        resized_arr = np.mean(np.array(resized_img), axis = 2)
    else:
        resized_arr = np.array(resized_img)
    # Crop chosen number of windows
    lag = int((int(resized_arr.shape[1]) - shape[1]) / nb_win -1)
    bounds = np.arange(0, int(resized_arr.shape[1]) - shape[1], lag)
    arrs_crop = list()
    for k in bounds:
        if greys:
            cropped_img = resized_arr[:, k:k+shape[1]].reshape(shape[0], shape[1], 1)
        else:
            cropped_img = resized_arr[:, k:k+shape[1]]
        arrs_crop.append(cropped_img)
    return arrs_crop

In [58]:
def data_augmentation(train_path, rotation_range = 10, width_shift_range = .2, height_shift_range = .2, shear_range = .2, zoom_range = .2, horizontal_flip = True, nb_img = 2):
    classes = os.listdir(train_path)
    datagen = ImageDataGenerator(
           rotation_range=rotation_range,
           width_shift_range=width_shift_range,
           height_shift_range=height_shift_range,
           shear_range=shear_range,
           zoom_range=zoom_range,
           horizontal_flip=horizontal_flip,
           fill_mode='nearest')

    for classe in classes:
        picts = os.listdir(train_path + '/' + classe)
        print(classe)        

        for pict in picts:
            img = Image.open(train_path + '/' + classe + '/' + pict)
            img = np.array(img)
            img = img.reshape((1,) + img.shape)
            i=0
            for batch in datagen.flow(img, batch_size=1, save_to_dir=train_path, save_prefix=classe + '/' + classe, save_format='jpg'):
                i += 1
                if i > nb_img:
                    break  # otherwise the generator would loop indefinitely

In [5]:
path = 'C:/Users/Lilian/Documents/Projet_deep_learning/G7_SEATGURU/Int/Airbus/'

In [60]:
data_augmentation(path + 'data_train')

A320
A321
A330
A350
A380


In [6]:
# Image data generator
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(path + 'data_train',
                                                    target_size=(224,224),
                                                    color_mode='rgb',
                                                    batch_size=64,
                                                    class_mode='categorical',
                                                    shuffle=True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_directory(path + 'data_test',
                                                   target_size=(224,224),
                                                   color_mode='rgb',
                                                   batch_size=64,
                                                   class_mode='categorical',
                                                   shuffle=True)

Found 4805 images belonging to 5 classes.
Found 537 images belonging to 5 classes.


# Transfer learning

In [7]:
# Create the base pre-trained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Add layers
x = base_model.output

x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)

x = Dense(1024)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Dense(512)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

# Last layer used to predict our classes
predictions = Dense(5, activation = 'softmax')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# Don't retrain pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=keras.optimizers.Adam(0.001),loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [8]:
# Reduce learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              patience=2, 
                              verbose=1, 
                              factor=0.5, 
                              min_lr=0.00001)

In [ ]:
step_size_train = 20 #train_generator.n // train_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    epochs=5,
                    validation_data=test_generator,
                    callbacks=[reduce_lr],
                    validation_steps=20)

Epoch 1/5
